In [6]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  4 23:28:32 2022

@author: parkh
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from SIM_gannt import *

learning_rate = 0.001  
gamma = 1
buffer_limit = 20000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(132,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
        self.number_of_time_list = np.array([1 for x in range(6)])
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        #print(out)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return act
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        return act,act_list
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users\parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
            
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
        s = env.reset()
        done = False
        score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        print(a_list)
        print(a)
        s_prime, r, done = env.step(a)
        print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
Flow_time, machine_util, util, makespan, score =main()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)
      
    

--------------------------------------------------
flow time: 4556, util : 0.902, makespan : 301
n_episode: 1, score : -171.0, n_buffer : 200, eps : 8.0%
--------------------------------------------------
flow time: 4543, util : 0.907, makespan : 307
n_episode: 2, score : -161.0, n_buffer : 300, eps : 8.0%
--------------------------------------------------
flow time: 4491, util : 0.901, makespan : 305
n_episode: 3, score : -171.0, n_buffer : 400, eps : 8.0%
--------------------------------------------------
flow time: 4763, util : 0.895, makespan : 313
n_episode: 4, score : -184.0, n_buffer : 500, eps : 8.0%
--------------------------------------------------
flow time: 4761, util : 0.904, makespan : 307
n_episode: 5, score : -170.0, n_buffer : 600, eps : 8.0%
--------------------------------------------------
flow time: 4646, util : 0.902, makespan : 299
n_episode: 6, score : -167.0, n_buffer : 700, eps : 7.9%
--------------------------------------------------
flow time: 4669, util : 0

--------------------------------------------------
flow time: 2913, util : 0.939, makespan : 267
n_episode: 54, score : -87.0, n_buffer : 5500, eps : 7.5%
--------------------------------------------------
flow time: 3140, util : 0.926, makespan : 283
n_episode: 55, score : -113.0, n_buffer : 5600, eps : 7.4%
--------------------------------------------------
flow time: 3270, util : 0.937, makespan : 296
n_episode: 56, score : -92.0, n_buffer : 5700, eps : 7.4%
--------------------------------------------------
flow time: 3621, util : 0.920, makespan : 272
n_episode: 57, score : -122.0, n_buffer : 5800, eps : 7.4%
--------------------------------------------------
flow time: 3459, util : 0.928, makespan : 299
n_episode: 58, score : -109.0, n_buffer : 5900, eps : 7.4%
--------------------------------------------------
flow time: 2947, util : 0.946, makespan : 268
n_episode: 59, score : -78.0, n_buffer : 6000, eps : 7.4%
--------------------------------------------------
flow time: 3196,

--------------------------------------------------
flow time: 3203, util : 0.914, makespan : 278
n_episode: 107, score : -124.0, n_buffer : 10800, eps : 6.9%
--------------------------------------------------
flow time: 3211, util : 0.932, makespan : 271
n_episode: 108, score : -100.0, n_buffer : 10900, eps : 6.9%
--------------------------------------------------
flow time: 3625, util : 0.906, makespan : 273
n_episode: 109, score : -146.0, n_buffer : 11000, eps : 6.9%
--------------------------------------------------
flow time: 3390, util : 0.917, makespan : 283
n_episode: 110, score : -126.0, n_buffer : 11100, eps : 6.9%
--------------------------------------------------
flow time: 3454, util : 0.938, makespan : 249
n_episode: 111, score : -90.0, n_buffer : 11200, eps : 6.9%
--------------------------------------------------
flow time: 3252, util : 0.915, makespan : 284
n_episode: 112, score : -129.0, n_buffer : 11300, eps : 6.9%
--------------------------------------------------
fl

--------------------------------------------------
flow time: 2739, util : 0.942, makespan : 290
n_episode: 159, score : -84.0, n_buffer : 16000, eps : 6.4%
--------------------------------------------------
flow time: 2777, util : 0.939, makespan : 288
n_episode: 160, score : -88.0, n_buffer : 16100, eps : 6.4%
--------------------------------------------------
flow time: 2716, util : 0.942, makespan : 296
n_episode: 161, score : -86.0, n_buffer : 16200, eps : 6.4%
--------------------------------------------------
flow time: 2572, util : 0.908, makespan : 284
n_episode: 162, score : -133.0, n_buffer : 16300, eps : 6.4%
--------------------------------------------------
flow time: 2663, util : 0.946, makespan : 287
n_episode: 163, score : -76.0, n_buffer : 16400, eps : 6.4%
--------------------------------------------------
flow time: 2766, util : 0.942, makespan : 289
n_episode: 164, score : -85.0, n_buffer : 16500, eps : 6.4%
--------------------------------------------------
flow t

--------------------------------------------------
flow time: 2734, util : 0.957, makespan : 301
n_episode: 212, score : -63.0, n_buffer : 20000, eps : 5.9%
--------------------------------------------------
flow time: 2727, util : 0.945, makespan : 270
n_episode: 213, score : -79.0, n_buffer : 20000, eps : 5.9%
--------------------------------------------------
flow time: 2745, util : 0.949, makespan : 270
n_episode: 214, score : -75.0, n_buffer : 20000, eps : 5.9%
--------------------------------------------------
flow time: 2530, util : 0.935, makespan : 269
n_episode: 215, score : -89.0, n_buffer : 20000, eps : 5.9%
--------------------------------------------------
flow time: 2692, util : 0.941, makespan : 273
n_episode: 216, score : -85.0, n_buffer : 20000, eps : 5.8%
--------------------------------------------------
flow time: 2531, util : 0.955, makespan : 262
n_episode: 217, score : -62.0, n_buffer : 20000, eps : 5.8%
--------------------------------------------------
flow ti

...

--------------------------------------------------
flow time: 2648, util : 0.958, makespan : 291
n_episode: 848, score : -60.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2619, util : 0.933, makespan : 276
n_episode: 849, score : -96.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 850, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2638, util : 0.951, makespan : 285
n_episode: 851, score : -69.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2668, util : 0.922, makespan : 292
n_episode: 852, score : -117.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2640, util : 0.946, makespan : 279
n_episode: 853, score : -76.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow t

--------------------------------------------------
flow time: 2764, util : 0.950, makespan : 307
n_episode: 901, score : -75.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 902, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2632, util : 0.930, makespan : 277
n_episode: 903, score : -100.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 904, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 905, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 906, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow t

--------------------------------------------------
flow time: 3054, util : 0.946, makespan : 301
n_episode: 954, score : -81.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2653, util : 0.954, makespan : 280
n_episode: 955, score : -64.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2595, util : 0.961, makespan : 273
n_episode: 956, score : -54.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 957, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2576, util : 0.967, makespan : 273
n_episode: 958, score : -46.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow time: 2891, util : 0.938, makespan : 288
n_episode: 959, score : -90.0, n_buffer : 20000, eps : 1.0%
--------------------------------------------------
flow ti

-3
[-25.93317032 -25.25827217 -27.34619522 -27.20015907 -24.64415359
 -24.80605316]
4
0
[-26.14229774 -25.3961525  -27.40854645 -27.46208    -25.14763641
 -25.09041405]
5
-3
[-24.77641678 -23.87716866 -25.84093475 -26.0150795  -23.89720917
 -23.5305748 ]
5
-1
[-23.9782505  -23.0923748  -24.74385452 -25.1129055  -23.4179039
 -22.86286545]
5
-2
[-21.95002937 -21.25506783 -22.7087059  -22.79494286 -21.15794373
 -20.93946838]
5
-3
[-22.70022392 -22.0155735  -23.29155159 -23.43559647 -21.86864853
 -21.53766251]
5
-2
[-22.88827705 -22.25429726 -23.4471035  -23.55666924 -21.97557068
 -21.72079086]
5
-4
[-22.51416016 -21.91779518 -23.13074112 -23.27207375 -21.54269028
 -21.31187248]
5
-4
[-19.04525948 -19.13288116 -20.04283524 -19.62122726 -17.45650291
 -18.27635193]
4
-3
[-17.24699974 -17.56369972 -18.45133972 -17.75393486 -15.46219444
 -16.7035408 ]
4
-3
[-16.89890671 -17.282444   -18.22934914 -17.34710693 -14.88773918
 -16.29187965]
4
-1
[-15.93081379 -16.34464073 -17.24069595 -16.40324974 

FlowTime: 2966
machine_util: 0
util: 0.9421157684630739
makespan: 296
Score -87.0
